In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import ToTensor
import torch.optim as optim

In [2]:
df = pd.DataFrame()

In [3]:
for i in range(1):
    df = pd.concat((df, pd.read_pickle(f"training_data.hkl.zip")), ignore_index=True)

In [4]:
class RMADataset(Dataset):
    def __init__(self, df, envs=50, prev_actions=20):
        self.data = df
        self.envs = envs
        self.prev_actions = prev_actions
        
    def __len__(self):
        return len(self.data)

    def get_row_data(self, idx):
        loc = self.data.iloc[idx]
        label = loc["extrinsics"]
        actions = loc["actions"]
        xt = loc["X"]
        orientation = loc["orientation"]
        
        return label, actions, xt, orientation
    
    def __getitem__(self, idx):
        label, actions, xt, orientation = self.get_row_data(idx)
        
        input_len = len(actions) + len(xt) + len(orientation)
        # placeholder prefilled with zeros
        data = [0.0] * self.prev_actions * input_len
        data[0:input_len] = actions + xt + orientation
        start_id = input_len

        for next_idx in range(idx - self.envs, max(0, idx - (self.prev_actions -1) * self.envs), -self.envs):
            _, actions, xt, orientation = self.get_row_data(next_idx)
            data[start_id:start_id + input_len] = actions + xt + orientation
            start_id += input_len
        
        return torch.tensor(label), torch.tensor([data])
        

In [5]:
dataset = RMADataset(df)

In [6]:
l, d = dataset[50]
d.shape

torch.Size([1, 780])

In [7]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        conv1 = nn.Conv1d(1, 32, 3, stride=3)
        conv2 = nn.Conv1d(32, 64, 5, stride=3)
        conv3 = nn.Conv1d(64, 128, 5, stride=2)

        conv4 = nn.Conv1d(128, 64, 7, stride=2)
        conv5 = nn.Conv1d(64, 64, 5, stride=2)
        conv6 = nn.Conv1d(64, 32, 5, stride=1)
        conv7 = nn.Conv1d(32, 32, 3, stride=1)
        
        self.layers = nn.Sequential(conv1, nn.BatchNorm1d(32), nn.ReLU(),
                                    conv2, nn.BatchNorm1d(64), nn.ReLU(),
                                    conv3, nn.BatchNorm1d(128), nn.ReLU(),
                                    conv4, nn.BatchNorm1d(64), nn.ReLU(),
                                    conv5, nn.BatchNorm1d(64), nn.ReLU(),
                                    conv6, nn.BatchNorm1d(32), nn.ReLU(),
                                    conv7, nn.Flatten())

    def forward(self, x):
        x = self.layers(x)
        return x

net = Net()

In [8]:
summary(net, (1, 780), device="cpu")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1              [-1, 32, 260]             128
       BatchNorm1d-2              [-1, 32, 260]              64
              ReLU-3              [-1, 32, 260]               0
            Conv1d-4               [-1, 64, 86]          10,304
       BatchNorm1d-5               [-1, 64, 86]             128
              ReLU-6               [-1, 64, 86]               0
            Conv1d-7              [-1, 128, 41]          41,088
       BatchNorm1d-8              [-1, 128, 41]             256
              ReLU-9              [-1, 128, 41]               0
           Conv1d-10               [-1, 64, 18]          57,408
      BatchNorm1d-11               [-1, 64, 18]             128
             ReLU-12               [-1, 64, 18]               0
           Conv1d-13                [-1, 64, 7]          20,544
      BatchNorm1d-14                [-1

In [9]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
net.to(device)
dataloader = DataLoader(dataset, batch_size=750, shuffle=True, num_workers=16, pin_memory=True)

In [10]:
criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [11]:
for epoch in range(20):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        
        labels, inputs = data
        labels = labels.to(device)
        inputs = inputs.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 200:.3f}')
            running_loss = 0.0
    print(f'Epoch {epoch} loss: {running_loss}')
    torch.save(net.state_dict(), f"adaptation_module_{epoch}.pth")
print(f'Finished Training {running_loss}')

Epoch 0 loss: 0.757495257537812
Epoch 1 loss: 0.26722390833310783
Epoch 2 loss: 0.20453171175904572
Epoch 3 loss: 0.17171470099128783
Epoch 4 loss: 0.15051976405084133
Epoch 5 loss: 0.134312309557572
Epoch 6 loss: 0.12206238997168839
Epoch 7 loss: 0.1114116805838421
Epoch 8 loss: 0.10211934265680611
Epoch 9 loss: 0.09581702668219805
Epoch 10 loss: 0.08916688617318869
Epoch 11 loss: 0.08458455698564649
Epoch 12 loss: 0.07851323310751468
Epoch 13 loss: 0.0741229047998786
Epoch 14 loss: 0.07154082698980346
Epoch 15 loss: 0.06844137888401747
Epoch 16 loss: 0.06458885793108493
Epoch 17 loss: 0.0619288042653352
Epoch 18 loss: 0.05819810589309782
Epoch 19 loss: 0.05717500182799995
Finished Training 0.05717500182799995
